In [1]:
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import discord
from discord.ext import commands

intents = discord.Intents.all()

client = commands.Bot(command_prefix = '!', intents = intents)

@client.command(name="delete")
async def delete(ctx):
    messages = await ctx.channel.history(limit=100)

    async for message in messages:
        if message.author == client.user:
            await message.delete()

@client.event
async def on_message(message):
    if message.author == client.user:
        return
    if message.content.startswith('!hello'):
        await message.channel.send('Hello!')

client.run("MTA5MTI1OTI5MzMxNzIxODMwNA.GK826G.63dgEjpDU6kMt_oa7xLNnjAgonjJOTXVAFddKo")

[2023-03-31 10:43:32] [INFO    ] discord.client: logging in using static token
[2023-03-31 10:43:32] [INFO    ] discord.client: logging in using static token
[2023-03-31 10:43:32] [INFO    ] discord.client: logging in using static token
[2023-03-31 10:43:32] [INFO    ] discord.client: logging in using static token
[2023-03-31 10:43:33] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: eedede9257f751e1785fef3bc6a6757a).
[2023-03-31 10:43:33] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: eedede9257f751e1785fef3bc6a6757a).
[2023-03-31 10:43:33] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: eedede9257f751e1785fef3bc6a6757a).
[2023-03-31 10:43:33] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: eedede9257f751e1785fef3bc6a6757a).
